In [40]:
import json

input_file = "last_letter_concatenation_training.json"  # Replace with the path to your JSONL file
output_file = "output.txt"  # Output file path

with open(input_file, "r") as f:
    data = json.load(f)

with open(output_file, "w") as f:
    for i in range(0,int(len(data)*0.7)):
        # json_data = json.loads(line.strip())
        prompt = data[i]["prompt"]
        completion = data[i]["reasoning"]
        answer = data[i]["answer"]
        f.write(f"[Q] {prompt}\n")
        # f.write("[Q] " + prompt + "\n")
        f.write(f"[A] {completion}--> {answer} END\n")
        # f.write("[A] " + completion + " --> " + answer  " END\n")
        f.write("\n")

# with open(output_file, "w") as f:
#     for question in data:
#         # json_data = json.loads(line.strip())
#         prompt = question["prompt"]
#         completion = question["reasoning"]
#         answer = question["answer"]
#         f.write(f"[Q] {prompt}\n")
#         # f.write("[Q] " + prompt + "\n")
#         f.write(f"[A] {completion}--> {answer} END\n")
#         # f.write("[A] " + completion + " --> " + answer  " END\n")
#         f.write("\n")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data

In [34]:
import pandas as pd
import numpy as np
import re
import os

In [41]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory():
    combined_text = ""
    file_path = 'output.txt'
    combined_text += read_txt(file_path)
    return combined_text

In [42]:
# train_directory = '/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/training_data/q_and_a'
text_data = read_documents_from_directory()
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters
text_data = re.sub(r"END", "END\n",text_data)
text_data

'[Q] Take the last letters of each words in "Whitney Erika Tj Benito" and concatenate them.\n[A] Sure! Here are the last letters of each name:\n* Whitney: y\n* Erika: a\n* Tj: j\n* Benito: o\nSo, the concatenation of the last letters is "yo".--> yajo END\n\n[Q] Take the last letters of each words in "Lucky Mireya Jj Kc" and concatenate them.\n[A] Sure, let\'s do that!\nThe last letters of each word in "Lucky Mireya Jj Kc" are:\n* Lucky: y\n* Mireya: a\n* Jj: j\n* Kc: c\nSo, the concatenation of the last letters is "yajjc".--> yajc END\n\n[Q] Take the last letters of each words in "Caleb Chase Eleazar Chanel" and concatenate them.\n[A] Sure, let\'s do that! Here are the last letters of each name:\n* Caleb: b\n* Chase: e\n* Eleazar: z\n* Chanel: l\nSo, the concatenation of the last letters of each name is "blel".--> berl END\n\n[Q] Take the last letters of each words in "Silvia Carolina Stan Chuck" and concatenate them.\n[A] Sure, let\'s do that! Here are the last letters of each name:\n

In [43]:
with open(output_file, 'w') as output_file:
    # output_file.write("")
    output_file.write(text_data)
    output_file.close()

In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [44]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [45]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [47]:
train_file_path = "output.txt"
model_name = 'gpt2'
output_dir = 'custom_q_and_a'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50
save_steps = 20000

In [12]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [11]:
import torch

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count())
print(torch.cuda.current_device())

cuda
1
0


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.679400
1000,0.211200
1500,0.118600
2000,0.075200
2500,0.054500


Checkpoint destination directory custom_q_and_a/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [50]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)

In [51]:
model2_path = "custom_q_and_a"
sequence2 = '[Q] Take the last letters of each words in "Lucky Mireya Jj Kc" and concatenate them.'
max_len = 110
answer = generate_text(model2_path, sequence2, max_len)
print(answer)

[Q] Take the last letters of each words in "Lucky Mireya Jj Kc" and concatenate them.
[A] Sure, let's do that!
The last letters of each word in "Lucky Mireya Jj Kc" are:
* Lucky: y
* Mireya: a
* Jj: j
* Kc: c
So, the concatenation of the last letters is "yajjc".--> yajc END

[Q


In [52]:
f = open('last_letter_concatenation_training.json')
test_data = json.load(f)

In [ ]:
correct_predictions = 0
model2_path = "custom_q_and_a"
max_len = 130
for i in range(int(len(test_data)*0.7), len(test_data)):

  new_prompt = test_data[i]['prompt']
  answer = generate_text(model2_path, "[Q] "+new_prompt, max_len)
  try:
    pred = str(answer.split("-->")[1].split("END")[0].strip())
    # print(answer.split("-->")[1].split("END")[0])
  except:
    pred = ""
    pass
  ans = test_data[i]['answer']
  if pred == ans:
    correct_predictions+= 1


In [55]:
total_data_points = len(test_data) - int(len(test_data)*0.7)
print("Accuracy = ",(correct_predictions/total_data_points)*100, "%")

Accuracy =  0.43859649122807015 %


In [ ]:
# for i in range(int(len(test_data)*0.7), len(test_data)):

#   new_prompt = test_data[i]['prompt']
#   answer = generate_text(model2_path, "[Q] "+new_prompt, max_len)
#   try:
#     pred = int(answer.split("-->")[1].split("END")[0].strip())
#     print(answer)
#     print(pred)
#     # break
#   except:
#     pred = -9999
#     pass
#   ans = test_data[i]['answer']
#   if '.' in test_data[i]['answer']:
#     pred=float(pred)
#     ans = float(ans)
#     print(ans)
#   else:
#     ans = int(ans)
#     print(ans)

#   break
#   if pred == ans:
#     correct_predictions+= 1